In [143]:
import PyPlot
#import Seaborn
import JLD2
import Distributions

In [2]:
include("src\\NGSIM.jl")

# td = load_trajdata(1);
JLD2.@load "td.jld" td
rd = NGSIM.ROADWAY_101

(S, id_lookup) = td_sparse(td)
;

update_params! (generic function with 1 method)

In [ ]:
fs = [
    s::VehicleState -> s.v * cos(s.posF.ϕ), 
    s::VehicleState -> s.v * sin(s.posF.ϕ),
    s::VehicleState -> length(rd[s.posF.roadind.tag.segment].lanes) - s.posF.roadind.tag.lane + 1
]

T = min(5000, nnz(S))

Y = AF64(length(fs), T)

for t in 1:T
    s = S.nzval[t]
    for (i, f) in fs |> enumerate
        @inbounds Y[i, t] = f(s)
    end
end

In [11]:
# the continuous data
YΓ = view(Y, 1:2, :)
# the discrete data (Int to use as indexing)
YΔ = round.(Int, Y[3, :])
NΓ = size(YΓ, 1)
NΔ = size(YΔ, 1)
;

In [14]:
#
# HMM Init
#

K = 5 # number states
M = 3 # number gaussian mixtures per state

# dimension of obs
N = NΓ + NΔ

# number of discrete observation states
# should only be 6 lanes in the data (but just in case)
L = 7

#
# transition parameters
#

# transition matrix; logs to avoid numberical underflow
# dirichlet returns distributions as the columns
A = rand(Dirichlet(K, 1), K)

# initial
π0 = rand(Dirichlet(K, 1))

#
# emission parameters
#

# bΔ[l, k] = P(yₜ = l | xₜ = k) (rows sum to 1)
# only for NΔ = 1; should be [rand(...)' for _ in 1:MΔ]
bΔ = rand(Dirichlet(L, 1), K)

# mixuter parameters for each MV per state
# c[m, k] is weight of m-th gaussian in for k-th state
c = rand(Dirichlet(M, 1), K)

# μs[:, m, k] is the mean of m-th mixture of the k-th state
μs = randn(NΓ, M, K) .+ squeeze(mean(YΓ, 2), 2)

# Σs[:, :, m, k] is the covariance of m-th mixture of the k-th state
Σs = repeat(cov(YΓ, 2), outer=(1, 1, M, K))
;

In [21]:
# number of trajectories
E = S.m

log_A = log.(A)
log_π0 = log.(π0)
log_bΔ = log.(bΔ)
log_c = log.(c)

invΣs = similar(Σs)
logdetΣs = similar(Σs, M, K)


@simd for m in 1:M
    for k in 1:K
        invΣs[:, :, m, k] = inv(Σs[:, :, m, k])
        logdetΣs[m, k] = logdet(Σs[:, :, m, k])
    end
end

# log P(y₁, ..., yₜ, xₜ | θ)
log_α = AF64(K, T)
# log P(yₜ₊₁, ..., y_T | xₜ =i, θ)
log_β = AF64(K, T)

# log P(yₜ | xₜ, θ)
log_b = similar(log_α)

# P(xₜ = j | Y, θ)
γ = AF64(K, T)
# P(Xₜ = j, Zⱼₜ = m| Y, θ) = prob of state j, and m-th mixture at t
γ_mix = AF64(M, K, T)
# P(xₜ = j, xₙ₋₁ = i | Y, θ)
ξ = AF64(K, K, T)
;

In [154]:
@inline function log_bΓ(m::Int, k::Int, t::Int)
    o = Vector{Float64}(NΓ)
    lpdf::Float64 = 0.0

    # shamelessly stolen from Distributions.jl/src/multivariate/mvnormal.jl
    lpdf = -(NΓ * log2π  + logdetΣs[m, k])/2
    o = YΓ[:, t] - μs[:, m, k]
    lpdf -= dot(o, (invΣs[:, :, m, k] * o)) / 2
    
    return lpdf
end

log_bΓ (generic function with 1 method)

In [345]:
# log_b[t, i] =  p(Yₜ = yₜ | Xₜ = i) = p(YΓₜ = yₜ | Xₜ = i) * p(YΔₜ = yₜ | Xₜ = i)
lgmm = AF64(M) # log of sum of GMM pdf

@inbounds for t = 1:T
    for k in 1:K # per state
        lgmm[:] = log_c[:, k]
        for m in 1:M # per mixture
            lgmm[m] += log_bΓ(m, k, t)
        end

        log_b[k, t] = logsumexp(lgmm)
    end

    log_b[:, t] .+= log_bΔ[YΔ[t], :]
end

In [110]:
# [ log_a[i, t-1] + log_A[j, i] ]ᵢ
temp = AF64(K)

log_α[:, 1] = log_π0 .+ log_b[:, 1]
@inbounds for t in 2:T
    for j in 1:K
        temp[:] = log_α[:, t-1] .+ log_A[j, :]
        log_α[j, t] = logsumexp(temp) + log_b[j, t]
    end
end

In [111]:
# backward pass
# temp is [ log_A[i, j] + log_β[j, t+1] + log_b[j, t+1] ]ⱼ

log_β[:, T] = 0
@inbounds for t = (T-1):-1:1
    for i in 1:K
        temp[:] = log_A[i, :] + log_b[:, t+1] + log_β[:, t+1]
        log_β[i, t] = logsumexp(temp)
    end
end

In [128]:
@inbounds for t in 1:T
    γ[:, t] = log_α[:, t] .+ log_β[:, t]
    γ[:, t] -= logsumexp(γ[:, t])
end
map!(exp, γ, γ)
;

In [127]:
@inbounds for t = 1:(T-1)
    ξ[:, :, t] = log_α[:, t]' .+ log_A .+ log_b[:, t+1] .+ log_β[:, t+1]
    ξ[:, : ,t] -= logsumexp(ξ[:, :, t])
end
map!(exp, ξ, ξ)
;

In [402]:
@inbounds for t in 1:T
    # work in logs for a bit
    γ_mix[:, :, t] = log_c
    γ_mix[:, :, t] .+= log.(γ[:, t])'
    γ_mix[:, :, t] .-= log_b[:, t]'
    # counter discrete prob 
    γ_mix[:, :, t] .+= log_bΔ[YΔ[t], :]'

    @inbounds for k in 1:K # per state
        for m in 1:M # per mixture
            γ_mix[m, k, t] += log_bΓ(m, k, t)
        end
    end
end
map!(exp, γ_mix, γ_mix)
;